## Pandas Snippets

In [60]:
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python_reference/master/Data/some_soccer_data.csv')
df.head()

,PLAYER,SALARY,GP,G,A,SOT,PPG,P
0,Sergio Agüero\n Forward — Manchester City,$19.2m,16.0,14,3.0,34,13.12,209.98
1,Eden Hazard\n Midfield — Chelsea,$18.9m,21.0,8,4.0,17,13.05,274.04
2,Alexis Sánchez\n Forward — Arsenal,$17.6m,NaN,12,7.0,29,11.19,223.86
3,Yaya Touré\n Midfield — Manchester City,$16.6m,18.0,7,1.0,19,10.99,197.91
4,Ángel Di María\n Midfield — Manchester United,$15.0m,13.0,3,NaN,13,10.17,132.23


In [61]:
# Converting column names to lowercase

df.columns = [c.lower() for c in df.columns]

# or
# df.rename(columns=lambda x : x.lower())

df.tail(3)

,player,salary,gp,g,a,sot,ppg,p
7,Cesc Fàbregas\n Midfield — Chelsea,$14.0m,20.0,2,14.0,10,10.47,209.49
8,Saido Berahino\n Forward — West Brom,$13.8m,21.0,9,0.0,20,7.02,147.43
9,Steven Gerrard\n Midfield — Liverpool,$13.8m,20.0,5,1.0,11,7.50,150.01


In [62]:
df = df.rename(columns={'p': 'points', 
                        'gp': 'games',
                        'sot': 'shots_on_target',
                        'g': 'goals',
                        'ppg': 'points_per_game',
                        'a': 'assists',})

df.tail(3)

,player,salary,games,goals,assists,shots_on_target,points_per_game,points
7,Cesc Fàbregas\n Midfield — Chelsea,$14.0m,20.0,2,14.0,10,10.47,209.49
8,Saido Berahino\n Forward — West Brom,$13.8m,21.0,9,0.0,20,7.02,147.43
9,Steven Gerrard\n Midfield — Liverpool,$13.8m,20.0,5,1.0,11,7.50,150.01


In [63]:
# Processing `salary` column

df['salary'] = df['salary'].apply(lambda x: x.strip('$m'))
df.tail()

,player,salary,games,goals,assists,shots_on_target,points_per_game,points
5,Santiago Cazorla\n Midfield — Arsenal,14.8,20.0,4,NaN,20,9.97,NaN
6,David Silva\n Midfield — Manchester City,14.3,15.0,6,2.0,11,10.35,155.26
7,Cesc Fàbregas\n Midfield — Chelsea,14.0,20.0,2,14.0,10,10.47,209.49
8,Saido Berahino\n Forward — West Brom,13.8,21.0,9,0.0,20,7.02,147.43
9,Steven Gerrard\n Midfield — Liverpool,13.8,20.0,5,1.0,11,7.50,150.01


In [64]:
#df['team'] = pd.Series('', index=df.index)

# or
df.insert(loc=8, column='position', value='') 

df.tail(3)

,player,salary,games,goals,assists,shots_on_target,points_per_game,points,position
7,Cesc Fàbregas\n Midfield — Chelsea,14.0,20.0,2,14.0,10,10.47,209.49,
8,Saido Berahino\n Forward — West Brom,13.8,21.0,9,0.0,20,7.02,147.43,
9,Steven Gerrard\n Midfield — Liverpool,13.8,20.0,5,1.0,11,7.50,150.01,


In [65]:
%timeit df['added1'] = pd.Series('', index=df.index)
#%time df.insert(loc=len(df.columns), column='added2', value='') 
%timeit df['added3'] = ''


196 µs ± 32.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
74.6 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [66]:
df.drop(['added1', 'added3'], axis=1, inplace=True)

,player,salary,games,goals,assists,shots_on_target,points_per_game,points,position
0,Sergio Agüero\n Forward — Manchester City,19.2,16.0,14,3.0,34,13.12,209.98,
1,Eden Hazard\n Midfield — Chelsea,18.9,21.0,8,4.0,17,13.05,274.04,
2,Alexis Sánchez\n Forward — Arsenal,17.6,NaN,12,7.0,29,11.19,223.86,
3,Yaya Touré\n Midfield — Manchester City,16.6,18.0,7,1.0,19,10.99,197.91,
4,Ángel Di María\n Midfield — Manchester United,15.0,13.0,3,NaN,13,10.17,132.23,
5,Santiago Cazorla\n Midfield — Arsenal,14.8,20.0,4,NaN,20,9.97,NaN,
6,David Silva\n Midfield — Manchester City,14.3,15.0,6,2.0,11,10.35,155.26,
7,Cesc Fàbregas\n Midfield — Chelsea,14.0,20.0,2,14.0,10,10.47,209.49,
8,Saido Berahino\n Forward — West Brom,13.8,21.0,9,0.0,20,7.02,147.43,
9,Steven Gerrard\n Midfield — Liverpool,13.8,20.0,5,1.0,11,7.50,150.01,


In [67]:
# Processing `player` column

def process_player_col(text):
    name, rest = text.split('\n')
    position, team = [x.strip() for x in rest.split(' — ')]
    return pd.Series([name, team, position])

df[['player', 'team', 'position']] = df.player.apply(process_player_col)

# modified after tip from reddit.com/user/hharison
#
# Alternative (inferior) approach:
#
#for idx,row in df.iterrows():
#    name, position, team = process_player_col(row['player'])
#    df.ix[idx, 'player'], df.ix[idx, 'position'], df.ix[idx, 'team'] = name, position, team
    
df.tail(3)

,player,salary,games,goals,assists,shots_on_target,points_per_game,points,position,added1,added3,team
7,Cesc Fàbregas,14.0,20.0,2,14.0,10,10.47,209.49,Midfield,,,Chelsea
8,Saido Berahino,13.8,21.0,9,0.0,20,7.02,147.43,Forward,,,West Brom
9,Steven Gerrard,13.8,20.0,5,1.0,11,7.50,150.01,Midfield,,,Liverpool


In [68]:
cols = ['player', 'position', 'team']
df[cols] = df[cols].applymap(lambda x: x.lower())
df.head()

,player,salary,games,goals,assists,shots_on_target,points_per_game,points,position,added1,added3,team
0,sergio agüero,19.2,16.0,14,3.0,34,13.12,209.98,forward,,,manchester city
1,eden hazard,18.9,21.0,8,4.0,17,13.05,274.04,midfield,,,chelsea
2,alexis sánchez,17.6,NaN,12,7.0,29,11.19,223.86,forward,,,arsenal
3,yaya touré,16.6,18.0,7,1.0,19,10.99,197.91,midfield,,,manchester city
4,ángel di maría,15.0,13.0,3,NaN,13,10.17,132.23,midfield,,,manchester united


In [69]:
import numpy as np

In [74]:
%%timeit

df.append(pd.Series(
                [np.nan]*len(df.columns), # Fill cells with NaNs
                index=df.columns),    
                ignore_index=True)

2.29 ms ± 98.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
%%timeit
nans = [np.nan] * len(df.columns)
df.loc[-1] = nans

3.89 ms ± 369 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
df

,player,salary,games,goals,assists,shots_on_target,points_per_game,points,position,added1,added3,team
0,sergio agüero,19.2,16.0,14.0,3.0,34.0,13.12,209.98,forward,,,manchester city
1,eden hazard,18.9,21.0,8.0,4.0,17.0,13.05,274.04,midfield,,,chelsea
2,alexis sánchez,17.6,NaN,12.0,7.0,29.0,11.19,223.86,forward,,,arsenal
3,yaya touré,16.6,18.0,7.0,1.0,19.0,10.99,197.91,midfield,,,manchester city
4,ángel di maría,15.0,13.0,3.0,NaN,13.0,10.17,132.23,midfield,,,manchester united
5,santiago cazorla,14.8,20.0,4.0,NaN,20.0,9.97,NaN,midfield,,,arsenal
6,david silva,14.3,15.0,6.0,2.0,11.0,10.35,155.26,midfield,,,manchester city
7,cesc fàbregas,14.0,20.0,2.0,14.0,10.0,10.47,209.49,midfield,,,chelsea
8,saido berahino,13.8,21.0,9.0,0.0,20.0,7.02,147.43,forward,,,west brom
9,steven gerrard,13.8,20.0,5.0,1.0,11.0,7.50,150.01,midfield,,,liverpool
